# Explorando el Tokenizador

En esta sección final, vamos a instanciar el tokenizador `GPT2TokenizerFast` utilizando los pesos de `Xenova/claude-tokenizer`.

El objetivo es analizar cómo el modelo "ve" el texto, prestando atención a:
1. El tamaño del vocabulario.
2. La diferencia entre listas y tensores.
3. El impacto de los espacios y la puntuación.

In [3]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('Xenova/claude-tokenizer')

# Verificamos el tamaño total del vocabulario
print(f"Tamaño del vocabulario: {tokenizer.vocab_size}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tamaño del vocabulario: 65000


## Codificación Básica

Diferencia entre obtener una lista de IDs estándar de Python y obtener Tensores de PyTorch (necesarios para alimentar el modelo).

In [4]:
# Codificación simple (lista de enteros)
print(tokenizer.encode('this is some text'))

# Salida completa del tokenizador
print(tokenizer('this is some text'))

# Salida en formato tensores de PyTorch
print(tokenizer('this is some text', return_tensors='pt'))

[3490, 365, 895, 1373]
{'input_ids': [3490, 365, 895, 1373], 'attention_mask': [1, 1, 1, 1]}
{'input_ids': tensor([[3490,  365,  895, 1373]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


## Palabras Multi-token y Espaciado

Aquí analizamos cómo el tokenizador maneja palabras que no están completas en su vocabulario y cómo **el espacio en blanco** es tratado como un carácter crítico.

Observa cómo `' hypothetical'` (con espacio) y `'hypothetical'` (sin espacio) generan tokens diferentes.

In [6]:
print("--- Ejemplos de palabras Multi-token y Espacios ---")
words = [ 'hypothetical',
          ' hypothetical',
          'hypothetical ',
          ' hypothteical'  ]

for word in words:
  toks = tokenizer.encode(word)
  print(f'"{word}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

words = [ "can't",
          " can't",
          "cant" ]

for word in words:
  toks = tokenizer.encode(word)
  print(f'"{word}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

words = [ ' straight forward',
          ' straightforward',
          ' straight-forward'  ]

for word in words:
  toks = tokenizer.encode(word)
  print(f'"{word}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

words = [ 'parttime',
          ' part-time',
          'part-time' ]

for word in words:
  toks = tokenizer.encode(word)
  print(f'"{word}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

--- Ejemplos de palabras Multi-token y Espacios ---
"hypothetical" tiene 2 tokens ([30678, 36881]):
   ['hypot', 'hetical']

" hypothetical" tiene 1 tokens ([44086]):
   [' hypothetical']

"hypothetical " tiene 3 tokens ([30678, 36881, 225]):
   ['hypot', 'hetical', ' ']

" hypothteical" tiene 4 tokens ([18806, 1309, 384, 709]):
   [' hyp', 'oth', 'te', 'ical']

"can't" tiene 2 tokens ([3152, 828]):
   ['can', "'t"]

" can't" tiene 2 tokens ([689, 828]):
   [' can', "'t"]

"cant" tiene 1 tokens ([25918]):
   ['cant']

" straight forward" tiene 2 tokens ([7638, 4258]):
   [' straight', ' forward']

" straightforward" tiene 1 tokens ([29388]):
   [' straightforward']

" straight-forward" tiene 3 tokens ([7638, 17, 6978]):
   [' straight', '-', 'forward']

"parttime" tiene 2 tokens ([1620, 793]):
   ['part', 'time']

" part-time" tiene 3 tokens ([919, 17, 793]):
   [' part', '-', 'time']

"part-time" tiene 3 tokens ([1620, 17, 793]):
   ['part', '-', 'time']



## Errores Ortográficos (Common Misspellings)

Una de las grandes ventajas de la tokenización por sub-palabras (BPE/WordPiece) es que no falla ante errores ortográficos. Simplemente rompe la palabra mal escrita en fragmentos más pequeños que sí conoce.

In [7]:
print("--- Errores Ortográficos ---")
words = [ ' accommodate acommodate',
          ' a lot alot',
          ' definately definitely',
          'occurrence occurence occurance']

for word in words:
  toks = tokenizer.encode(word)
  print(f'"{word}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

--- Errores Ortográficos ---
" accommodate acommodate" tiene 4 tokens ([27029, 1163, 17293, 353]):
   [' accommodate', ' ac', 'ommod', 'ate']

" a lot alot" tiene 4 tokens ([269, 3949, 460, 331]):
   [' a', ' lot', ' al', 'ot']

" definately definitely" tiene 3 tokens ([39887, 2704, 14159]):
   [' defin', 'ately', ' definitely']

"occurrence occurence occurance" tiene 6 tokens ([48275, 2535, 540, 11970, 8265, 634]):
   ['occurrence', ' occ', 'ure', 'nce', ' occur', 'ance']



## Puntuación y Estilo Literario

El tokenizador es literal. Las comillas curvas (“...”) típicas de los libros o procesadores de texto son tokens diferentes a las comillas rectas ("...") de programación.

In [8]:
print("--- Puntuación ---")
sentences = [ "People say, you shouldn't do that.",
              "People say; you shouldn't do that.",
              "People say -- you shouldn't do that.",  ]

for senten in sentences:
  toks = tokenizer.encode(senten)
  print(f'"{senten}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')


text = '“Curiouser and curiouser!” cried Alice (she was so much surprised, that for the moment she quite forgot how to speak good English).'
toks = tokenizer.encode(text)
print(f'"{text}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

sentences = [ '“No, no!” said the Queen. “Sentence first—verdict afterwards.”',
              '“No, no!” said the Queen. “Sentence first - verdict afterwards.”']

for senten in sentences:
  toks = tokenizer.encode(senten)
  print(f'"{senten}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

--- Puntuación ---
"People say, you shouldn't do that." tiene 9 tokens ([9308, 2236, 16, 583, 12329, 828, 705, 427, 18]):
   ['People', ' say', ',', ' you', ' shouldn', "'t", ' do', ' that', '.']

"People say; you shouldn't do that." tiene 9 tokens ([9308, 2236, 31, 583, 12329, 828, 705, 427, 18]):
   ['People', ' say', ';', ' you', ' shouldn', "'t", ' do', ' that', '.']

"People say -- you shouldn't do that." tiene 9 tokens ([9308, 2236, 1786, 583, 12329, 828, 705, 427, 18]):
   ['People', ' say', ' --', ' you', ' shouldn', "'t", ' do', ' that', '.']

"“Curiouser and curiouser!” cried Alice (she was so much surprised, that for the moment she quite forgot how to speak good English)." tiene 32 tokens ([3675, 39, 4427, 304, 400, 329, 1803, 21293, 400, 25912, 16905, 32268, 344, 3647, 504, 779, 1935, 14843, 16, 427, 351, 279, 4152, 970, 5055, 14759, 1369, 317, 6006, 1929, 6075, 673]):
   ['“', 'C', 'uri', 'ou', 'ser', ' and', ' cur', 'iou', 'ser', '!”', ' cried', ' Alice', ' (', 'she', ' w

## Tokenización de Código

El código (LaTeX, Python, MATLAB) es denso en tokens. Caracteres como `\`, `{`, `}`, `[` suelen tokenizarse individualmente, lo que explica por qué el contexto de la IA se llena más rápido con código que con lenguaje natural.

In [9]:
print("--- Código ---")
sentences = [ r'\textbf{Leibniz} & ', # latex (usamos r'' para raw strings en python)
              r'&= \lim_{h\to0}\alpha \left[\frac{f(x+h)-f(x)}{h}\right] \;+\; \lim_{h\to0} \beta \left[\frac{g(x+h)-g(x)}{h}\right]', # latex
              'diffVects = targetActs[layeri,:,:,1] - targetActs[layeri-1,:,:,1]; diffNorms[layeri,1] = np.linalg.norm(diffVects,axis=1).mean()', # python
              '[Y,X] = meshgrid(linspace(-4,4,21)); G = exp( -(X.^2+Y.^2)/10 );' # MATLAB
              ]

for senten in sentences:
  toks = tokenizer.encode(senten)
  print(f'"{senten}" tiene {len(toks)} tokens ({toks}):\n   {[tokenizer.decode(t) for t in toks]}\n')

--- Código ---
"\textbf{Leibniz} & " tiene 10 tokens ([64, 57619, 95, 2447, 606, 82, 461, 97, 1273, 225]):
   ['\\', 'textbf', '{', 'Le', 'ib', 'n', 'iz', '}', ' &', ' ']

"&= \lim_{h\to0}\alpha \left[\frac{f(x+h)-f(x)}{h}\right] \;+\; \lim_{h\to0} \beta \left[\frac{g(x+h)-g(x)}{h}\right]" tiene 61 tokens ([64977, 669, 3961, 1151, 76, 64, 535, 20, 2015, 2741, 669, 1903, 8824, 3498, 95, 74, 12, 92, 15, 76, 4547, 74, 12, 92, 27681, 76, 2015, 1233, 65, 57037, 7372, 31, 669, 3961, 1151, 76, 64, 535, 20, 97, 669, 3814, 669, 1903, 8824, 3498, 95, 75, 12, 92, 15, 76, 4547, 75, 12, 92, 27681, 76, 2015, 1233, 65]):
   ['&=', ' \\', 'lim', '_{', 'h', '\\', 'to', '0', '}\\', 'alpha', ' \\', 'left', '[\\', 'frac', '{', 'f', '(', 'x', '+', 'h', ')-', 'f', '(', 'x', ')}{', 'h', '}\\', 'right', ']', ' \\;', '+\\', ';', ' \\', 'lim', '_{', 'h', '\\', 'to', '0', '}', ' \\', 'beta', ' \\', 'left', '[\\', 'frac', '{', 'g', '(', 'x', '+', 'h', ')-', 'g', '(', 'x', ')}{', 'h', '}\\', 'right', ']']

"diffVe

# Análisis de Resultados: ¿Cómo "piensa" el Tokenizador?

A través de los ejemplos anteriores, hemos observado comportamientos críticos que definen cómo los Modelos de Lenguaje (LLMs) procesan la información antes de entenderla. A continuación, desglosamos los hallazgos clave:

### 1. El Espacio es Información (Sensibilidad al Espaciado)
Al observar la palabra `hypothetical`, notamos que las variantes con espacios producen tokens diferentes.
* **Observación:** `'hypothetical'` (sin espacio) tiene un ID diferente a `' hypothetical'` (con espacio inicial).
* **Por qué ocurre:** En tokenizadores modernos (como el de GPT-2/3/4 o Claude), el espacio se suele fusionar con la palabra siguiente (representado a veces como `Ġ` o `_`).
* **Implicación:** Un error de espaciado en un *prompt* puede alterar sutilmente la representación numérica que recibe el modelo.

### 2. Palabras Compuestas y el "Efecto Guion"
Comparando `parttime`, `part-time` y ` part-time`:
* **Observación:** El guion (`-`) actúa frecuentemente como un "cortador".
* **Análisis:** Mientras que una palabra compuesta común podría tener su propio token único, al introducir un guion forzamos al tokenizador a dividirla en sus componentes semánticos (`part`, `-`, `time`). Esto aumenta el conteo de tokens (el "costo" computacional) pero mantiene la claridad semántica.

### 3. Robustez ante Errores (Misspellings)
Este es uno de los puntos más fuertes de la tokenización por sub-palabras (*Subword Tokenization*).
* **Caso:** `acommodate` (incorrecto) vs `accommodate` (correcto).
* **Comportamiento:** El modelo **no** arroja un error ni devuelve un token de "desconocido" (`<UNK>`). En su lugar, descompone la palabra mal escrita en fragmentos más pequeños que sí reconoce (ej. `a`, `com`, `mo`, `date`).
* **Ventaja:** Esto permite al modelo "leer" y entender palabras mal escritas basándose en su fonética o morfología, aproximando el significado original.

### 4. Puntuación: Lo Literario vs. Lo Técnico
El experimento con *Alicia en el País de las Maravillas* revela la literalidad del modelo.
* **Hallazgo:** Las comillas curvas o tipográficas (`“ ”`), comunes en novelas y PDFs, son tokens totalmente distintos a las comillas rectas (`" "`) usadas en programación.
* **Implicación:** Un texto copiado de un libro antiguo puede tokenizarse de forma menos eficiente que un texto digital moderno, debido al uso de caracteres "raros" o estilizados.

### 5. La "Densidad" del Código (Code Tokenization)
Al tokenizar LaTeX o Python, el número de tokens se dispara.
* **El Fenómeno:** El lenguaje natural (Inglés, Español) tiene una relación *palabra:token* cercana a 1:1 o 0.75:1. Sin embargo, el código tiene una densidad mucho mayor.
* **Causa:** Los símbolos de sintaxis (`\`, `{`, `}`, `[`, `]`, `=`, `;`) raramente se agrupan. Casi cada símbolo cuenta como un token independiente.
* **Conclusión Crítica:** Procesar código consume la "ventana de contexto" del modelo mucho más rápido que procesar texto narrativo.